In [50]:
import daisy
import numpy as np
from collections import defaultdict

In [89]:
fragments = '/mnt/hdd2/DATA/block-face-EM/Megalopta/predictions/megalopta_FB_setup_03_0_2_FBOK_600000_1/megalopta_FB_setup_03_0_2_FBOK_600000_1_frags.zarr'
chunk_size = [5000, 5000, 5000]
comp_chunk_scale = 2

In [117]:
ds = daisy.open_ds(fragments, 'frags')
chunk_shape = daisy.Coordinate(chunk_size) / daisy.Coordinate(ds.voxel_size)
n_chunks = np.ceil(np.array(ds.shape)/np.array(chunk_shape)).astype(int)

chunk_list = []
chunk_groups = []
for ix, x in enumerate(range(n_chunks[2])):
    for iy, y in enumerate(range(n_chunks[1])):
        for iz, z in enumerate(range(n_chunks[0])):
            chunk = (x,y,z)
            chunk_list.append(chunk)
            group = tuple((np.array([ix,iy,iz])) // comp_chunk_scale)
            chunk_groups.append(group)

In [91]:
len(chunk_groups[(0,0,0)])

8

In [125]:
chunk_coords = [4,4,4]
chunk_groups = np.array(chunk_groups)
chunk_list = np.array(chunk_list)

In [122]:
adj_chunk_coords_list = np.empty([6,3], dtype=int)
i=0
for d in [-1,1]:
    for dim in range(3):
        diff = np.zeros([3], dtype=int)
        diff[dim] = d
        adj_chunk_coords_list[i,:] = chunk_coords + diff
        i += 1
adj_chunk_coords_list = adj_chunk_coords_list[np.all(adj_chunk_coords_list >= 0, 1)]

In [127]:
main_group = chunk_groups[np.all(chunk_list == chunk_coords, 1)]
main_group

array([[2, 2, 2]])

In [128]:
mask = np.zeros(len(chunk_list)).astype(bool)
for ch in adj_chunk_coords_list:
    mask = mask | np.all(ch == np.array(chunk_list), 1)
    
np.array(chunk_groups)[mask]

array([[1, 2, 2],
       [2, 1, 2],
       [2, 2, 1],
       [2, 2, 2],
       [2, 2, 2],
       [2, 2, 2]])

In [130]:
chunks_to_cut = np.array(chunk_list)

In [171]:
adj_chunk_coords_list = np.empty([6,3], dtype=int)
i=0
for d in [-1,1]:
    for dim in range(3):
        diff = np.zeros([3], dtype=int)
        diff[dim] = d
        adj_chunk_coords_list[i,:] = chunk_coords + diff
        i += 1
adj_chunk_coords_list = adj_chunk_coords_list[np.all(adj_chunk_coords_list >= 0, 1)]

# Is chunk to be isolated?
if np.any(np.all(chunk_coords == chunks_to_cut, 1)):
    # If yes, simply isolate it from all chunks around
    mask = np.zeros(adj_chunk_coords_list.shape[0], dtype=bool)
else:
    # If not isolate from any chunk that needs to be isolated...
    mask = np.array([np.logical_not(np.any(np.all(chunk == chunks_to_cut, 1))) for chunk in adj_chunk_coords_list])

# Always keep all chunks in the same group connected to each other
main_group = chunk_groups[np.all(chunk_list == chunk_coords, 1)]
for i, chunk in enumerate(adj_chunk_coords_list):
    mask[i] = np.all(chunk_groups[np.all(chunk == np.array(chunk_list), 1)] == main_group, 1)

In [172]:
mask

array([False, False, False,  True,  True,  True])

In [170]:
mask

array([False, False, False,  True,  True,  True])